In [5]:
import pandas as pd
from datetime import timedelta

# --------------------------------------------
# Paramètres
# --------------------------------------------
TOLERANCE_SECONDS = 5  # tolérance de fusion temporelle
wifi_ap1_file = "wifi_log2_ap1.jsonl"
wifi_ap2_file = "wifi_log2_ap2.jsonl"
server_ap1_file = "trafic_metrics_server1.jsonl"
server_ap2_file = "trafic_metrics_server2.jsonl"

# --------------------------------------------
# Fonction de préparation d'un AP
# --------------------------------------------
def prepare_ap(wifi_file, server_file, suffix):
    # Charger Wi-Fi (jsonl)
    df_wifi = pd.read_json(wifi_file, lines=True)

    # Supprimer les lignes sans throughput (pas de test en cours)
    if "throughput_bps" in df_wifi.columns:
        df_wifi = df_wifi[df_wifi["throughput_bps"].notnull()].copy()

    # Aplatir la liste "clients"
    if "clients" in df_wifi.columns:
        df_exp = df_wifi.explode("clients")
        clients = pd.json_normalize(df_exp["clients"])
    else:
        df_exp = df_wifi.copy()
        clients = pd.DataFrame()

    # Colonnes globales
    global_cols = [c for c in df_exp.columns if c != "clients"]
    global_data = df_exp[global_cols].reset_index(drop=True)
    clients = clients.reset_index(drop=True)

    df_flat = pd.concat([global_data, clients], axis=1)

    # Charger serveur (trafic reçu)
    df_server = pd.read_json(server_file, lines=True)

    # Fusion via traffic_label
    if "traffic_label" in df_flat.columns and "traffic_label" in df_server.columns:
        df_flat = pd.merge(
            df_flat,
            df_server,
            on="traffic_label",
            how="left",
            suffixes=("", "_server")
        )

    # Convertir timestamps
    # côté Wi-Fi : "timestamp" ; côté serveur : "timestamp_server" éventuellement
    if "timestamp" in df_flat.columns:
        df_flat["timestamp"] = pd.to_datetime(df_flat["timestamp"], errors="coerce")
    if "timestamp_server" in df_flat.columns:
        df_flat["timestamp_server"] = pd.to_datetime(df_flat["timestamp_server"], errors="coerce")

    # Ajouter suffixe aux colonnes (sauf colonne de fusion temporelle)
    # On garde une colonne 'timestamp_<suffix>' pour clarté
    df_flat = df_flat.rename(columns={"timestamp": f"timestamp_{suffix}"})
    # Ajouter suffixe à toutes les autres colonnes sauf traffic_label (qu’on garde pour référence)
    cols_to_rename = {
        c: f"{c}_{suffix}"
        for c in df_flat.columns
        if c not in [f"timestamp_{suffix}", "traffic_label"]
    }
    df_flat = df_flat.rename(columns=cols_to_rename)

    return df_flat

# --------------------------------------------
# Préparer AP1 et AP2
# --------------------------------------------
ap1 = prepare_ap(wifi_ap1_file, server_ap1_file, "ap1")
ap2 = prepare_ap(wifi_ap2_file, server_ap2_file, "ap2")

# --------------------------------------------
# Fusion temporelle horizontale
# --------------------------------------------
# On utilise merge_asof avec tolérance
# Il faut trier par timestamp
ap1_sorted = ap1.sort_values(by="timestamp_ap1").reset_index(drop=True)
ap2_sorted = ap2.sort_values(by="timestamp_ap2").reset_index(drop=True)

# merge_asof fusionne en appariant chaque ligne AP1 avec la plus proche AP2
merged = pd.merge_asof(
    ap1_sorted,
    ap2_sorted,
    left_on="timestamp_ap1",
    right_on="timestamp_ap2",
    direction="nearest",
    tolerance=pd.Timedelta(seconds=TOLERANCE_SECONDS)
)

# --------------------------------------------
# Option : filtrer les lignes sans correspondance (si désiré)
# (les lignes où aucune correspondance AP2 dans la tolérance auront NaN)
merged_valid = merged.dropna(subset=["timestamp_ap2"]).copy()

# --------------------------------------------
# Sauvegarde
# --------------------------------------------
merged_valid.to_csv("dual_ap_horizontal_dataset.csv", index=False, encoding="utf-8")
print(f"Fusion réalisée : {len(merged_valid)} lignes appariées (tolérance {TOLERANCE_SECONDS}s)")


✅ Fusion réalisée : 7 lignes appariées (tolérance 5s)
